<a href="https://colab.research.google.com/github/sandhrabijoy/_/blob/master/Pytorch_practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np,pandas as pd ,pylab as pl
import h5py,torch
from tensorflow import image as timage
from torchvision import transforms,utils
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn as tnn


In [8]:
from IPython.core.magic import register_line_magic
dev=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
class TData(tds):
  def __init__(self,X,y):
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.int32)
  def __getitem__(self,index):
    train_img,train_lbl=self.X[index],self.y[index]
    return train_img,train_lbl
  def __len__(self):
    return self.y.shape[0]

In [12]:
class VGG19(tnn.Module):
  def __init__(self,num_classes):
    super(VGG19,self).__init__()
    self.block1=tnn.Sequential(
        tnn.Conv2d(in_channels=3,out_channels=64,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )
    self.block2=tnn.Sequential(
        tnn.Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=128,out_channels=128,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )
    self.block3=tnn.Sequential(
        tnn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )
    self.block4=tnn.Sequential(
        tnn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )
    self.block5=tnn.Sequential(
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),stride=(1,1),padding=1),
        tnn.ReLU(),
        tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )

    self.classifier=tnn.Sequential(
        tnn.Linear(512,4096),
        tnn.ReLU(True),
        tnn.Linear(4096,4096),
        tnn.ReLU(True),
        tnn.Linear(4096,num_classes)
    )
    for m in self.modules():
      if isinstance(m,torch.nn.Conv2d):
        m.weight.detach().normal_(0,.05)
        if m.bias is not None:
          m.bias.detach().zero_()
      elif isinstance(m,torch.nn.Linear):
        m.weight.detach().normal_(0,.05)
        m.bias.detach().detach().zero_()
  def forward(self,x):
    x=self.block1(x)
    x=self.block2(x)
    x=self.block3(x)
    x=self.block4(x)
    x=self.block5(x)

    logits=self.classifier(x.view(-1,512))
    probs=tnn.functional.softmax(logits,dim=1)
    return logits,probs